In [ ]:
import os
import gc

import pandas as pd
import gspread_dataframe as gs
import pyperclip as clip # copiar o texto clipboard
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup

import random

# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")

#import sistemas
import sei
import functions
import page
from page import *
import helpers
import sistemas

% reload_ext autoreload
% autoreload 2

In [ ]:
def extrai_pares_tabulação(source):
    trs = source.find_all('tr')
    dados = {}
    i = 1
    for tr in trs:        
        td = tr.find_all('td', string=True)
        label = tr.find_all('label', string=True)
        
        i = 1
        for field, result in zip(td, label):
            field, result = field.text[:-1], result.text
            if field in dados:
                field = field + "_" + str(i + 1)
            dados[field] = result 
       
    return dados

In [ ]:
processo = r'53504.006463/2018-16'

In [ ]:
auth = functions.authenticate_google("files/anatel.json")
wb = auth.open(title="Banco_de_Dados_Cassacao")
sht = wb.get_worksheet(0)
df = gs.get_as_dataframe(sht, dtype=str)

In [77]:
usr = "rsilva"
pwd = "Savorthemom3nts"

page = webdriver.Ie()
scpx = sistemas.Scpx(page)

In [ ]:
validade_rf = sht.range('T210:T227')
fistel = sht.range('C210:C227')

In [ ]:
scpx.imprimir_licenca("33576590854")

In [ ]:
#dados = list()
for f, v in zip(fistel, validade_rf):
    dados = scpx.extrai_cadastro(f.value, tipo_id='id_fistel')
    
    i = input()
    
    if i == 'l':
        
        try:
        
            scpx.imprimir_licenca(f.value())
            
        except: next
        
    elif i == 'g':
        
        try:
                    
            scpx.movimento_cancelar(f.value)
            
        except:
            
            next
        
    
        
    #v.value = dados[-1].get('Validade Radiofreqüência', "")
    #sht.update_cell(v.row, v.col, v.value)
    #scpx.consulta(f.value, tipo_id="id_fistel")
    
    gc.collect()

In [78]:
scpx.movimento_cancelar("05712320860")

In [ ]:
df = pd.DataFrame(columns=dados[0].keys())

In [ ]:
for dic in dados:

    df = df.append(pd.Series(dic), ignore_index=True)
    
df.head()

In [ ]:
vals = []

for i, values in df[df["Processo"] == processo].iterrows():
    fistel = values.FISTEL
    while len(fistel) < 11:
        fistel = '0' + fistel
    scpx.consulta(fistel,tipo_id='id_fistel')
    source = soup(scpx.driver.page_source, "lxml")
    dados = extrai_pares_tabulação(source)
    
    scpx.wait_for_element_to_click((By.ID, "botaoFlatPróximo")).click()
    val_rf = source.find(id='labelDataValidadeRadioFrequencia')
    
    #source = soup(scpx.driver.page_source, "lxml")
    #dados = {**dados, **extrai_pares_tabulação(source)}
                                               
    #proximo = scpx.wait_for_element_to_click((By.ID, "botaoFlatEstação"))
    #proximo.click()
    
    #source = soup(scpx.driver.page_source, "lxml")
    #dados = {**dados, **extrai_pares_tabulação(source)}
    
    print("Nome: {}".format(values['NOME']))
    print("Validade de Radiofrequência: {}".format(val_rf.text))
    print("Data de Validade Licença: {}\n".format(dados['Data Validade Licença']))
    
    vals.append(val_rf) 
    
df.loc[df["Processo"] == processo, "Validade_RF"] = vals